In [61]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm
import matplotlib.pyplot as plt

In [84]:
matrice_ttc=pd.DataFrame({'depuis':['a','b','c'],'a':[0.8,0.7,0.6],'b':[0.17,0.255,0.330],'c':[0.03,0.045,0.07]})
hrc=list(matrice_ttc.columns)
m_ttc=np.matrix(matrice_ttc.iloc[:,1:])
pd_base=np.array([0.05,0.055,0.06])
pd_adv=np.array([0.055,0.075,0.08])
ead_pit=np.array([300,600,100])

In [85]:
pd_ttc=float(np.average(m_ttc[:,-1],0,ead_pit))

In [86]:
def correlation_baloise(pde):
    a=0.03*(1-math.exp(-35*pde))/(1-math.exp(-35))
    b=0.16*(1-(1-math.exp(-35*pde))/(1-math.exp(-35)))
    return (a+b)

corr=correlation_baloise(pd_ttc)

In [87]:
def z_macro(pd_pit,pd_ttc,corr):
    a=math.sqrt(1-corr)*norm.ppf(pd_pit)
    b=norm.ppf(pd_ttc)
    return ((a-b)/math.sqrt(corr))

[[0.03 ]
 [0.045]
 [0.07 ]]


array([[0.03485355],
       [0.0523264 ],
       [0.08130705]])

In [119]:
#Calcul du vecteur de defaut avec en entrée
#la PD TTC par CHR (CHR,1)
#la correlation (1,1)
#le facteur de risque systematique de l'année concernée (1,1)
def merton_default(pd_ttc,corr,z):
    a=norm.ppf(pd_ttc)+math.sqrt(corr)*z
    b=math.sqrt(1-corr)
    return norm.cdf(a/b)
print(m_ttc[:,-1])
merton_default(m_ttc[:,-1],corr,zbase[0])
#Pour une CHR d'une année donnée, calcul les taux de transition entre avec en entrée
#i et j CHR de départ et CHR d'arrivée
#la matrice de transition TTC (CHR,CHR)
#le vecteur de risque systématique (1,1)
#la matrice de transition PIT under construction (CHR,CHR)
def transition_merton(i,j,tm_ttc,corr,z,tm_pit):
    tm_ttc_cum=tm_ttc[i,j:].sum()
    if tm_ttc_cum>=1:
        temp=1-tm_pit[i,j+1:].sum()
    else:
        a=(norm.ppf(tm_ttc_cum)+math.sqrt(corr)*z)/math.sqrt(1-corr)
        temp=norm.cdf(a)-tm_pit[i,j+1:].sum()
    return temp

#Construction de la matrice de transition projetée avec en entrée
#Matrice de transition TTC (CHR,CHR)
#le facteur de correlation (1,1)
#le facteur de risque systématique (1,1)
def transition_matrix_merton(tm_ttc,corr,z):
    nb_rating=tm_ttc.shape[0]
    tm=np.matrix(np.zeros((nb_rating,nb_rating)))

    tm[:-1,-1]=merton_default(tm_ttc[:-1,-1],corr,z)
    tm[nb_rating-1,nb_rating-1]=1
    for i in range(nb_rating):
        for j in reversed(range(nb_rating-1)):
            if i == nb_rating-1:
                tm[i,j]=0
            else:
                tm[i,j]=transition_merton(i,j,tm_ttc,corr,z,tm)
    return tm

In [121]:
zbase=z_macro(pd_base,pd_ttc,corr)
zadv=z_macro(pd_adv,pd_ttc,corr)

mproj={'1base':transition_matrix_merton(m_ttc,corr,zbase[0]),
       '2base':transition_matrix_merton(m_ttc,corr,zbase[1]),
       '3base':transition_matrix_merton(m_ttc,corr,zbase[2]),
      '1adv':transition_matrix_merton(m_ttc,corr,zadv[0]),
       '2adv':transition_matrix_merton(m_ttc,corr,zadv[1]),
       '3adv':transition_matrix_merton(m_ttc,corr,zadv[2])}

In [123]:
mproj

{'1base': matrix([[0.77114798, 0.19399846, 0.03485355],
         [0.66116416, 0.28650944, 0.0523264 ],
         [0.        , 0.        , 1.        ]]),
 '2base': matrix([[0.75677691, 0.20462143, 0.03860166],
         [0.6439294 , 0.29856265, 0.05750794],
         [0.        , 0.        , 1.        ]]),
 '3base': matrix([[0.74297767, 0.21464395, 0.04237838],
         [0.62762226, 0.30969341, 0.06268432],
         [0.        , 0.        , 1.        ]]),
 '1adv': matrix([[0.75677691, 0.20462143, 0.03860166],
         [0.6439294 , 0.29856265, 0.05750794],
         [0.        , 0.        , 1.        ]]),
 '2adv': matrix([[0.70447347, 0.24166226, 0.05386428],
         [0.58329114, 0.33852313, 0.07818573],
         [0.        , 0.        , 1.        ]]),
 '3adv': matrix([[0.69246538, 0.24979403, 0.05774059],
         [0.56979817, 0.34685735, 0.08334447],
         [0.        , 0.        , 1.        ]])}